In [1]:
# https://github.com/airaria/TextBrewer/blob/master/examples/notebook_examples/sst2.ipynb

# TextBrewer -> https://aclanthology.org/2020.acl-demos.2.pdf 

In [2]:
import torch
device='cuda' if torch.cuda.is_available() else 'cpu' 

In [3]:
# !pip install transformers==4.8.2
# !pip install datasets
# !pip install textbrewer

In [4]:
import os
import torch
from transformers import BertForSequenceClassification, BertTokenizer,BertConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,load_metric
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
from sklearn import preprocessing

2022-05-01 21:02:30.301412: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 21:02:30.301455: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
train = pd.read_csv('trac1-dataset/english_train/english/agr_en_train.csv',names=['ID','sentence','label'])
val = pd.read_csv('trac1-dataset/english_train/english/agr_en_dev.csv',names=['ID','sentence','label'])
test = pd.read_csv('trac1-dataset/trac-gold-set/agr_en_fb_gold.csv',names=['ID','sentence','label'])

# To make binary class
train = train.replace(to_replace =["OAG","CAG"],value ="AG")
val = val.replace(to_replace =["OAG","CAG"],value ="AG")
test = test.replace(to_replace =["OAG","CAG"],value ="AG")

In [5]:
import re
import emot.emo_unicode as emo_unicode
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from string import digits

def emoToText(text):
    for em in emo_unicode.UNICODE_EMOJI:
        text = text.replace(em,''.join(emo_unicode.UNICODE_EMOJI[em]).replace("_"," ").replace(":",""))
    text = text.lower()
    return text

In [6]:
def english_preprocessing(text_list):
    
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;].')
#    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    punc = '''()-[]{};:'"\,<>./@#$%^&*_~'''
    
    regrex_pattern = re.compile(pattern = "["
    "\U0001F1E0-\U0001F1FF"  
    "\U0001F300-\U0001F5FF"  
    "\U0001F680-\U0001F6FF"  
    "\U0001F700-\U0001F77F"  
    "\U0001F780-\U0001F7FF"  
    "\U0001F800-\U0001F8FF"  
    "\U0001F900-\U0001F9FF"  
    "\U0001FA00-\U0001FA6F"  
    "\U0001FA70-\U0001FAFF"  
    "\U00002702-\U000027B0"  
    "\U000024C2-\U0001F251" 
    "]+", flags = re.UNICODE)
    
    for i in range(len(text_list)):
        
        text=text_list[i]   
        text = emoToText(text)
#        text=emoji.demojize(text)
        text = ' '.join(word for word in text.split() if word not in STOPWORDS)           
        text = re.sub(r"http\S+", "", text)          
        text = re.sub('@[^\s]+','',text)
        
        remove_digits = str.maketrans('', '', digits) 
        text = text.translate(remove_digits) 

        for ele in text:  
            if ele in punc:  
                text = text.replace(ele, "") 
        text=re.sub('\s+',' ',text)
        text.strip()
    
        text= regrex_pattern.sub(r'',text)
    
        text = REPLACE_BY_SPACE_RE.sub(' ', text)
        
        if text != text:
            text_list[i]='garbage'
        else:
            text_list[i]=text
    return text_list

In [7]:
def text_preprocessing(df):
    df1 = df
    df1.fillna("Garbage")
    df1.Text=df1.sentence.astype(str)
    text_list=df1['sentence'].to_list()
    text_list=english_preprocessing(text_list)  
    df1['sentence'] = text_list
    return df1

In [9]:
train = text_preprocessing(train)
val = text_preprocessing(val)
test = text_preprocessing(test)

/home/mane1/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


In [10]:
train.to_pickle('preprocessed_data/train.pkl')
val.to_pickle('preprocessed_data/val.pkl')
test.to_pickle('preprocessed_data/test.pkl')

In [5]:
train = pd.read_pickle('preprocessed_data/train.pkl')
val = pd.read_pickle('preprocessed_data/val.pkl')
test = pd.read_pickle('preprocessed_data/test.pkl')

In [6]:
# 1 = NAG (non aggresive), 0 = AG (aggresive)

le = preprocessing.LabelEncoder()
le.fit(train.label)

train['label'] = le.transform(train['label'])
val['label'] = le.transform(val['label'])
test['label'] = le.transform(test['label'])

#le.inverse_transform(test['label'])

In [7]:
train_dataset = Dataset(pa.Table.from_pandas(train))
val_dataset = Dataset(pa.Table.from_pandas(val))
test_dataset = Dataset(pa.Table.from_pandas(test))

In [8]:
train_dataset = train_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_dataset = val_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
test_dataset = test_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_dataset = val_dataset.remove_columns(['label'])
test_dataset = test_dataset.remove_columns(['label'])
train_dataset = train_dataset.remove_columns(['label'])

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
#, attention_probs_dropout_prob=0.3,hidden_dropout_prob=0.3)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
MAX_LENGTH = 128
train_dataset = train_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
val_dataset = val_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
test_dataset = test_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [12]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [13]:
#start training
training_args = TrainingArguments(          
    #output directory
    learning_rate=1e-4,
    num_train_epochs=5,              
    per_device_train_batch_size=32,                #batch size per device during training
    per_device_eval_batch_size=32,                #batch size for evaluation
    output_dir='results',
    logging_dir='logs',            
    logging_steps=100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=True,
    evaluation_strategy="steps")

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
    compute_metrics=compute_metrics)

train_out = trainer.train()

#after training, you could find traing logs and checpoints in your own dirve. also you can reset the file address in training args

torch.save(model.state_dict(), 'agg_teacher_model.pt')

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, sentence.
***** Running training *****
  Num examples = 11999
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1875
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mane (use `wandb login --relogin` to force relogin)
2022-05-01 21:03:07.619444: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 21:03:07.619478: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.603300,0.579379,0.731090,0.700292,0.741587,0.696064
200,0.568800,0.525658,0.748417,0.740063,0.740153,0.739975
300,0.564800,0.513579,0.755082,0.744235,0.747780,0.741950
400,0.505400,0.566193,0.757414,0.739791,0.757009,0.734358
500,0.422800,0.541318,0.756748,0.739173,0.756125,0.733792
600,0.430200,0.508527,0.749750,0.740987,0.741532,0.740493
700,0.393100,0.536049,0.751749,0.743186,0.743599,0.742803
800,0.328100,0.705731,0.758414,0.739532,0.760048,0.733734
900,0.253000,0.611353,0.745085,0.737319,0.736818,0.737883
1000,0.224800,0.678870,0.759414,0.745612,0.754856,0.741454


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, sentence.
***** Running Evaluation *****
  Num examples = 3001
  Batch size = 32
Saving model checkpoint to results/checkpoint-100
Configuration saved in results/checkpoint-100/config.json
Model weights saved in results/checkpoint-100/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, sentence.
***** Running Evaluation *****
  Num examples = 3001
  Batch size = 32
Saving model checkpoint to results/checkpoint-200
Configuration saved in results/checkpoint-200/config.json
Model weights saved in results/checkpoint-200/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, sentence.
***** Running Evaluation *****


# Start distillation

In [22]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=32) #prepare dataloader

In [23]:
import textbrewer
from textbrewer import GeneralDistiller
from textbrewer import TrainingConfig, DistillationConfig
from transformers import BertForSequenceClassification, BertConfig, AdamW,BertTokenizer
from transformers import get_linear_schedule_with_warmup

In [25]:
bert_config_T3 = BertConfig.from_json_file('student_config/bert_config_L3.json')#相对路径
bert_config_T3.output_hidden_states = True

student_model = BertForSequenceClassification(bert_config_T3) #, num_labels = 2
student_model.to(device=device)


bert_config = BertConfig.from_json_file('student_config/bert_config.json')
bert_config.output_hidden_states = True
teacher_model = BertForSequenceClassification(bert_config) #, num_labels = 2
teacher_model.load_state_dict(torch.load('agg_teacher_model.pt'))
teacher_model.to(device=device)

In [17]:
num_epochs = 20
num_training_steps = len(train_dataloader) * num_epochs
# Optimizer and learning rate scheduler
optimizer = AdamW(student_model.parameters(), lr=1e-4)

scheduler_class = get_linear_schedule_with_warmup
# arguments dict except 'optimizer'
scheduler_args = {'num_warmup_steps':int(0.1*num_training_steps), 'num_training_steps':num_training_steps}


def simple_adaptor(batch, model_outputs):
    return {'logits': model_outputs.logits, 'hidden': model_outputs.hidden_states}

distill_config = DistillationConfig(
    intermediate_matches=[    
     {'layer_T':0, 'layer_S':0, 'feature':'hidden', 'loss': 'hidden_mse','weight' : 1},
     {'layer_T':8, 'layer_S':2, 'feature':'hidden', 'loss': 'hidden_mse','weight' : 1}])
train_config = TrainingConfig()

distiller = GeneralDistiller(
    train_config=train_config, distill_config=distill_config,
    model_T=teacher_model, model_S=student_model, 
    adaptor_T=simple_adaptor, adaptor_S=simple_adaptor)


with distiller:
    distiller.train(optimizer, train_dataloader, num_epochs, scheduler_class=scheduler_class, scheduler_args = scheduler_args, callback=None)

In [18]:
test_model = BertForSequenceClassification(bert_config_T3)

In [19]:
test_model.load_state_dict(torch.load('saved_models/gs7500.pkl')) #gs4210 is the distilled model weights file

<All keys matched successfully>

In [20]:
from torch.utils.data import DataLoader
eval_dataloader = DataLoader(val_dataset, batch_size=8)

In [21]:
metric= load_metric("accuracy")
test_model.eval()
for batch in eval_dataloader:
    batch = {k: v for k, v in batch.items()}
    with torch.no_grad():
        outputs = test_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.7484171942685771}

wandb: Network error (ReadTimeout), entering retry loop.
